In [2]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from requests_html import HTML

________________________________________

**Bấm để đồng ý cookies**

In [3]:
# Đồng ý cookies
def accept_cookies(browser):
    cookies = browser.find_element(By.XPATH,"//button[@id='onetrust-accept-btn-handler']")
    cookies.click()

**Kéo xuống đến cuối trang với số lần dựa theo limit (số tracks muốn parse tối đa)**

In [4]:
def scroll_down(browser, limit):
    # số tracks hiện lên mỗi lần scroll đến cuối trang
    tracks_per_scroll = 10
    # số lần scroll xuống cuối trang
    scrolls = limit // tracks_per_scroll
    for i in range(scrolls - 1):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(3)

**Lấy tất cả users và tracks**

In [16]:
def get_all_users_tracks(browser):
    soup = BeautifulSoup(browser.page_source)
    songs = soup.find_all("li", "compactTrackList__item")
    for song in songs:
        user = song.find("span", "compactTrackListItem__user")
        if user is not None:
            if user not in users:
                users.append(user.get_text().strip())
        track = song.find("span", "compactTrackListItem__trackTitle").get_text().strip()
        if track not in tracks["track"]:
            tracks["track"].append(track)
            tracks["user"].append(user)

**Lấy tất cả playlists dựa theo limit**

In [6]:
def get_all_playlists(browser):
    soup = BeautifulSoup(browser.page_source)
    # trích xuất playlist
    posts = soup.find_all("li", "searchList__item")
    for post in posts:
        user = post.find("span", "soundTitle__usernameText").get_text().strip()
        playlist = post.find("a", "soundTitle__title").get_text().strip()
        if user not in users:
            users.append(user)
        playlists["user"].append(user)
        playlists["playlist"].append(playlist)

**Chuyển từ list sang csv/tsv**

In [7]:
def list_to_tsv_csv(listItem, outputFile):
    df = pd.DataFrame(listItem)
    df.to_csv(outputFile, sep='\t')

**Main**

In [11]:
def main(limit):
    alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    for letter in alphabet:
        url = "https://soundcloud.com/search/sets?q=" + letter
        browser = webdriver.Chrome()
        browser.get(url)
        
        time.sleep(5)
        accept_cookies(browser)
        
        time.sleep(1)
        scroll_down(browser, limit)
        
        time.sleep(1)
        get_all_users_tracks(browser)
        get_all_playlists(browser)
        
        time.sleep(1)
        browser.quit()

**Khởi tạo các list để lưu thông tin**

In [9]:
# Lưu user
users = []
# Lưu track
tracks = {"track":[], "user":[]}
# Lưu playlist
# playlists = {"playlist":[], "user":[], "tracks":[]}
playlists = {"playlist":[], "user":[]}

In [18]:
limit = 10
main(limit)

**Lưu data vào file tsv**

In [21]:
outputFile = "Crawl_data/user1.csv"
list_to_tsv_csv(users, outputFile)
outputFile = "Crawl_data/track1.csv"
list_to_tsv_csv(tracks, outputFile)
outputFile = "Crawl_data/playlist1.csv"
list_to_tsv_csv(playlists, outputFile)

In [20]:
print(len(users))
print(len(tracks["track"]))
print(len(playlists["playlist"]))

2327
2148
520
